In [261]:
import json
import pandas as pd
import numpy as np
import random

In [262]:
data = np.array(pd.read_csv('test2.csv'),dtype= int)
T,L = data.shape[0],data.shape[1]
values = [4,3,2,1]
M,A,E =0,0,0

In [263]:
def varSelection(array,N,D,j):

    if j==D:
        return 0,j

    l =0
    unassigned = []
    for l in range(0,N):
        if(array[l][j]==0):
            unassigned.append(l)
    
    if len(unassigned) == 0:
        return varSelection(array,N,D,j+1)

    for k in unassigned:
        if j>0 and (array[k][j-1] == 1 or array[k][j-1]==3): #if previous day morning or evening
            return k,j


    return random.choice(unassigned),j

In [264]:
def valueSelection(array,N,D,i,j,availableVal,preferedVal):

    if j>=1 and (array[i][j-1] == 1 or array[i][j-1]==3):   #if last day morning or even

        if preferedVal !=1 and availableVal[preferedVal]:
            return preferedVal

        elif availableVal[2]==1:
            return 2            #Giving Afternoon

        elif availableVal[4]==1 :
            return 4            #Giving Rest
        
        
        elif availableVal[3]==1:
            return 3            #atlast evening

        else:
            return -1
            
    elif availableVal[preferedVal]:     # returning prefered value
        return preferedVal

    for i in range(1,5):            #else any remaining value
        if availableVal[i]==1:
            return i
    
    return -1
    
    
    

In [265]:
def checkValid(array,i,j,val,m,a,e,r,N):

    if val == 1 and (m ==0 or(j>=1 and  (array[i][j-1]==1 or array[i][j-1]==3))):
        return False

    if val ==2 and a==0:
        return False

    if val==3 and e ==0:
        return False
        
    if val ==4 and r==0:
        return False
    
    if i==N-1:
        if val==1 and (a!=0 or e!=0):
            return False
        elif val==2 and (m!=0 or e!=0):
            return False
        elif val==3 and (a!=0 or m!=0):
            return False

    if j%7 ==6 and val!=4:
        chec =0
        for k in range(0,6):
            if array[i][j-k]==4:
                chec+=1
        if chec ==0:
            return False
    return True

In [266]:
def preferVal(m,a,e):
    
    d = {1:m,2:a,3:e}

    sortedDict = sorted(d.items(),key=lambda x:x[1],reverse=True)
    for i in sortedDict:
        return(i[0])

In [267]:
def NurseSol(array,N,D,m,a,e,r):
    
    sol = nurseSolver(array,0,0,N,D,m,a,e,r)
    if sol:
        return array
    return np.zeros((N,D))

In [268]:
def validfurther(array,j,m,a,e,N):
    
    if j==0:
        return True

    count = 0
    total = 0
    for k in range(0,N):
        if array[k][j]==0:
            total+=1
            if array[k][j-1] == 1 or array[k][j-1] == 3:
                count+=1
    
    if total-count < m:
        return False
    
    return True

In [269]:
def nurseSolver(array,i,j,N,D,m,a,e,r):

    if(j==D):
        return True
    
    if not validfurther(array,j,m,a,e,N):
        return False

    
    valAvailable = [1,1,1,1,1]
    
    for x in range(0,4):

        y = preferVal(m,a,e)    #which is most remaining
        
        value = valueSelection(array,N,D,i,j,valAvailable,y)

        if(value==-1):      
            return False
        
        valAvailable[value]=0

        if checkValid(array,i,j,value,m,a,e,r,N):
            result = False

            array[i][j] =value

            if value ==1:
                m-=1
            elif value ==2:
                a-=1
            elif value==3:
                e-=1 
            elif value ==4:
                r-=1
            

            # I,J = varSelection(array,N,D,j)
            
            # if J != j:
            #     result = nurseSolver(array,I,j+1,N,D,M,A,E,R)

            # else:
            #     result = nurseSolver(array,I,j,N,D,m,a,e,r)

            if i == N-1:
                result = nurseSolver(array,0,j+1,N,D,M,A,E,R)
            else:
                 result = nurseSolver(array,i+1,j,N,D,m,a,e,r)
            
            if result:
                return result
            else:
                array[i][j]=0
                if value ==1:
                    m+=1
                elif value ==2:
                    a+=1
                elif value==3:
                    e+=1
                elif value==4:
                    r+=1
    return False
        

In [270]:

if(L==5):
    for i in range (0,T):
        N,D,M,A,E = data[i]
        R = N-M-A-E
        array = np.zeros((N,D))

        if(M+A+E>=N):
            print("NO SOLUTION")
            continue
        solu = NurseSol(array,N,D,M,A,E,R)
        
        soln_list = []
        for p in range (0,D):
            dic = {}
            for q  in range(0,N):
                k = 'N'+str(q)+'_'+str(p)
                if(solu[q][p]==1):
                    dic[k]='M'
                elif(solu[q][p]==2):
                    dic[k]='A'
                elif(solu[q][p]==3):
                    dic[k]='E'
                else:
                    dic[k]='R'
                
            soln_list.append(dic)
        
        with open('solution.json','w') as file:
            for d in soln_list:
                json.dump(d,file)
                file.write("\n")

        

KeyboardInterrupt: 